In [56]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq


In [57]:
df = pd.read_parquet('dataset_renomeado.parquet')

# Tratamento dos dados para criação dos gráficos no streamlit

In [58]:
df.columns

Index(['id_ncontro', 'id_paciente', 'id_hospital', 'idade', 'imc',
       'cirurgia_eletiva', 'etnia', 'genero', 'altura', 'fonte_admissao_uti',
       'id_uti', 'tipo_estadia_uti', 'tipo_uti', 'dias_de_permanencia_pre_uti',
       'peso', 'diagnostico_pache_2', 'diagnostico_apache_3j',
       'apache_pos_operatorio', 'arf_apache', 'gcs_olhos_apache',
       'gcs_motor_apache', 'gcs_incapaz_apache', 'gcs_verbal_pache',
       'frequencia_cardiaca_apache', 'intubado_apache', 'map_apache',
       'frequencia_respiratoria_pache', 'temperatura_apache',
       'ventilado_apache', 'd1_pressao_arterial_diastolica_maxima',
       'd1_pressao_arterial_diastolica_minima',
       'd1_pressao_arterial_diastolica_nao_invasiva_maxima',
       'd1_pressao_arterial_diastólica_nao_invasiva_minima',
       'd1_frequencia_cardiaca_maxima', 'd1_frequencia_cardiaca_minima',
       'd1_pressao_arterial_media_maxima', 'd1_pressao_arterial_media_minima',
       'd1_pressao_arterial_media_nao_invasiva_maxima',

In [59]:
df.rename(columns={'h1_pressão_arterial_diastolica_maxima': 'h1_pressao_arterial_diastolica_maxima','h1_ressao_arterial_iastolica_minima':'h1_pressao_arterial_iastolica_minima' }, inplace=True)

In [2]:
#com certeza não vou usar essas colunas
colunas_deletar = ['id_ncontro', 'id_paciente', 'id_hospital','id_uti']
df = df.drop(columns=colunas_deletar)

## Tratar valores nulos 



In [64]:
for coluna in df.columns:
    if df[coluna].dtype == object:  
        df[coluna].fillna('Desconhecido', inplace=True)  
    else:
        media = df[coluna].mean()  
        df[coluna].fillna(media, inplace=True) 


## Converterndo para inteiro 

In [66]:
for coluna in df.columns:
    if df[coluna].dtype == float:  
          df[coluna] = df[coluna].replace({'0.': 0, '1.': 1}).astype(int)
    

## Organizando nomes

In [67]:
df['genero'] = df['genero'].replace({'M': 'Masculino', 'F': 'Feminino'}).fillna('Desconhecido')

In [68]:
df['tipo_estadia_uti'] = df['tipo_estadia_uti'].replace({'admit': 'Admitido', 'readmit': 'Readmitido', 'transfer': 'Transferido'})

In [69]:
fonte = {
    'Floor': 'Piso',
    'Accident & Emergency': 'Acidente e Emergência',
    'Operating Room / Recovery': 'Sala de Cirurgia / Recuperação',
    'Other Hospital': 'Outro Hospital',
    'Other ICU': 'Outra UTI'
}

df['fonte_admissao_uti'] = df['fonte_admissao_uti'].replace(fonte)

In [70]:
etnia = {
    'Caucasian': 'Caucasiano',
    'Hispanic': 'Hispanico',
    'African American': 'Afro-americano',
    'Asian': 'Asiático',
    'Native American': 'Indígena',
    'Other/Unknown': 'Outro/Desconhecido'
}

df['etnia'] = df['etnia'].replace(etnia)

In [71]:
tipo_uti = {
    'CTICU': 'CTI Cardíaco',
    'Med-Surg ICU': 'CTI Med-Cirúrgico',
    'CCU-CTICU': 'CTI Coronariano',
    'Neuro ICU': 'CTI Neurológico',
    'MICU': 'CTI Médico',
    'SICU': 'CTI Cirúrgico',
    'Cardiac ICU': 'CTI Cardíaco',
    'CSICU': 'CTI Cardiovascular'
}

df['tipo_uti'] = df['tipo_uti'].replace(tipo_uti)

In [72]:
df.to_parquet('dataset_streamlit.parquet')

In [73]:
df.columns = [coluna.replace("d1_", "") + "_primeiras_24_horas" if coluna.startswith("d1_") else
              coluna.replace("h1_", "") + "_primeira_hora" if coluna.startswith("h1_") else
              coluna for coluna in df.columns]


In [75]:
colunas_primeira_hora = [coluna.replace('_primeira_hora', '') for coluna in df.columns if '_primeira_hora' in coluna]
colunas_primeiras_24_horas = [coluna.replace('_primeiras_24_horas', '') for coluna in df.columns if '_primeiras_24_horas' in coluna]

colunas_em_comum = set(colunas_primeira_hora).intersection(colunas_primeiras_24_horas)


In [76]:
colunas_em_comum

{'frequencia_cardiaca_maxima',
 'frequencia_respiratoria_maxima',
 'frequencia_respiratoria_minima',
 'pressao_arterial_diastolica_maxima',
 'pressao_arterial_diastolica_nao_invasiva_maxima',
 'pressao_arterial_media_maxima',
 'pressao_arterial_media_minima',
 'pressao_arterial_media_nao_invasiva_maxima',
 'pressao_arterial_media_nao_invasiva_minima',
 'pressao_arterial_sistolica_maxima',
 'pressao_arterial_sistolica_minima',
 'pressao_arterial_sistolica_nao_invasiva_maxima',
 'spO2_maximo',
 'spO2_minimo'}

In [77]:
primeira_hora = ['h1_pressão_arterial_diastolica_maxima',
       'h1_ressao_arterial_iastolica_minima',
       'h1_pressao_arterial_diastolica_nao_invasiva_maxima',
       'h1_pressao_arterial_diastolica_nao_invasiva_minima',
       'h1_frequencia_cardiaca_maxima', 'h1_requencia_cardiaca_minima',
       'h1_pressao_arterial_media_maxima', 'h1_pressao_arterial_media_minima',
       'h1_pressao_arterial_media_nao_invasiva_maxima',
       'h1_pressao_arterial_media_nao_invasiva_minima',
       'h1_frequencia_respiratoria_maxima',
       'h1_frequencia_respiratoria_minima', 'h1_spO2_maximo', 'h1_spO2_minimo',
       'h1_pressao_arterial_sistolica_maxima',
       'h1_pressao_arterial_sistolica_minima',
       'h1_pressao_arterial_sistolica_nao_invasiva_maxima',
       'h1_pressão_arterial_sistolica_nao_invasiva_minima']

In [78]:
['idade', 'imc', 'cirurgia_eletiva', 'etnia', 'genero', 'altura','aids', 'cirrose', 'diabetes_mellitus',
       'insuficiencia_hepatica', 'imunossupressao', 'leucemia', 'linfoma','tumor_solido_com_metastase',]

['idade',
 'imc',
 'cirurgia_eletiva',
 'etnia',
 'genero',
 'altura',
 'aids',
 'cirrose',
 'diabetes_mellitus',
 'insuficiencia_hepatica',
 'imunossupressao',
 'leucemia',
 'linfoma',
 'tumor_solido_com_metastase']

In [79]:
['fonte_admissao_uti', 'tipo_estadia_uti', 'tipo_uti',
       'dias_de_permanencia_pre_uti', 'peso', 'diagnostico_pache_2',
       'diagnostico_apache_3j', 'apache_pos_operatorio', 'arf_apache',
       'gcs_olhos_apache', 'gcs_motor_apache', 'gcs_incapaz_apache',
       'gcs_verbal_pache', 'frequencia_cardiaca_apache', 'intubado_apache',
       'map_apache', 'frequencia_respiratoria_pache', 'temperatura_apache',
       'ventilado_apache','sistema_corporal_apache_3j','sistema_corporal_apache_2', 'morte_hospital']

['fonte_admissao_uti',
 'tipo_estadia_uti',
 'tipo_uti',
 'dias_de_permanencia_pre_uti',
 'peso',
 'diagnostico_pache_2',
 'diagnostico_apache_3j',
 'apache_pos_operatorio',
 'arf_apache',
 'gcs_olhos_apache',
 'gcs_motor_apache',
 'gcs_incapaz_apache',
 'gcs_verbal_pache',
 'frequencia_cardiaca_apache',
 'intubado_apache',
 'map_apache',
 'frequencia_respiratoria_pache',
 'temperatura_apache',
 'ventilado_apache',
 'sistema_corporal_apache_3j',
 'sistema_corporal_apache_2',
 'morte_hospital']

In [80]:
colunas_primeiras_24_horas = ['d1_pressao_arterial_diastolica_maxima',
       'd1_pressao_arterial_diastolica_minima',
       'd1_pressao_arterial_diastolica_nao_invasiva_maxima',
       'd1_pressao_arterial_diastólica_nao_invasiva_minima',
       'd1_frequencia_cardiaca_maxima', 'd1_frequencia_cardiaca_minima',
       'd1_pressao_arterial_media_maxima', 'd1_pressao_arterial_media_minima',
       'd1_pressao_arterial_media_nao_invasiva_maxima',
       'd1_pressao_arterial_media_nao_invasiva_minima',
       'd1_frequencia_respiratoria_maxima',
       'd1_frequencia_respiratoria_minima', 'd1_spO2_maximo', 'd1_spO2_minimo',
       'd1_pressao_arterial_sistolica_maxima',
       'd1_pressao_arterial_sistolica_minima',
       'd1_pressao_arterial_sistolica_nao_invasiva_maxima',
       'd1_pressao_arterial_sistolica_nao_invasiva_minima',
       'd1_temperatura_maxima', 'd1_temperatura_minima','d1_glicose_maxima', 
       'd1_glicose_minima', 'd1_potassio_maximo',
       'd1_potassio_minimo']

In [81]:
table = pa.Table.from_pandas(df)
pq.write_table(table, 'dataset_streamlit.parquet')

In [90]:
correlation_matrix = dados.corr()

colunas_correlacao_maior_04 = correlation_matrix[correlation_matrix['morte_hospital'] > 0.2].index.tolist()

colunas_correlacao_maior_04

/tmp/ipykernel_17243/3482270238.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = dados.corr()


['ventilado_apache', 'morte_hospital']